In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
X=np.load('./XValues1.npy')
Y=np.load('./YValues1.npy')
Y=np.squeeze(Y)

x_train_raw = X
y_train_raw = Y

Xtest=np.load('./XValues2.npy')
Ytest=np.load('./YValues2.npy')
Ytest=np.squeeze(Ytest)

x_test_raw = Xtest
y_test_raw = Ytest

# length check
assert(len(x_train_raw) == len(y_train_raw))
assert(len(x_test_raw) == len(y_test_raw))

In [3]:
n_classes = 3 # 3 labels: 0:normal, 1:MCI, 2:Alzh

x_train = np.reshape(x_train_raw,(-1,256,256,170,1))
x_test = np.reshape(x_test_raw,(-1,256,256,170,1))

# choosing a 16 by 16 by 16 slot in the images
x_train = x_train[:, 100:116, 100:116, 100:116, :]
x_test = x_test[:, 100:116, 100:116, 100:116, :]

# change lables to 1-hot form matrix
y_train = to_categorical(y_train_raw, n_classes)
y_test = to_categorical(y_test_raw, n_classes)


In [4]:
x_input = tf.placeholder(tf.float32, shape=[None, 16, 16, 16, 1])
y_input = tf.placeholder(tf.float32, shape=[None, n_classes]) 

In [5]:
def cnn_model(x_train_data, keep_rate=0.7, seed=None):
    
    # conv => 16*16*16
    conv1 = tf.layers.conv3d(inputs=x_train_data, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => 16*16*16
    conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => 8*8*8
    pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2)

    # conv => 8*8*8
    conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => 8*8*8
    conv5 = tf.layers.conv3d(inputs=conv4, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => 4*4*4
    pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2)

    # conv => 4*4*4
    conv7 = tf.layers.conv3d(inputs=pool6, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => 4*4*4
    conv8 = tf.layers.conv3d(inputs=conv7, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => 2*2*2
    pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2)

    cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, training=True)

    flattening = tf.reshape(cnn3d_bn, [-1, 2*2*2*128])
    dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu)
    # (1-keep_rate) is the probability that the node will be kept
    dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True)

    y_conv = tf.layers.dense(inputs=dropout, units=3)
    
    return y_conv

In [6]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, learning_rate=0.05, keep_rate=0.7, epochs=10, batch_size=128):

    prediction = cnn_model(x_input, keep_rate, seed = 1)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input))


    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
           
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    iterations = int(len(x_train_data)/batch_size) + 1
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        import datetime

        start_time = datetime.datetime.now()

        iterations = int(len(x_train_data)/batch_size) + 1
        # run epochs
        for epoch in range(epochs):
            start_time_epoch = datetime.datetime.now()
            print('Epoch', epoch, 'started', end='')
            epoch_loss = 0
            # mini batch
            for itr in range(iterations):
                mini_batch_x = x_train_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y = y_train_data[itr*batch_size: (itr+1)*batch_size]
                _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
                epoch_loss += _cost

            #  using mini batch in case not enough memory
            acc = 0
            itrs = int(len(x_test_data)/batch_size) + 1
            for itr in range(itrs):
                mini_batch_x_test = x_test_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y_test = y_test_data[itr*batch_size: (itr+1)*batch_size]
                acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x_test, y_input: mini_batch_y_test})
            end_time_epoch = datetime.datetime.now()
            print(' Testing Set Accuracy:',acc/itrs, ' Time elapse: ', str(end_time_epoch - start_time_epoch))
        end_time = datetime.datetime.now()
        print('Time elapse: ', str(end_time - start_time))

In [7]:
train_neural_network(x_train, y_train, x_test, y_test, epochs=10, batch_size=3, learning_rate=0.0001)

Epoch 0 started Testing Set Accuracy: 0.166666671634  Time elapse:  0:00:02.447662
Epoch 1 started Testing Set Accuracy: 0.333333335817  Time elapse:  0:00:02.408019
Epoch 2 started Testing Set Accuracy: 0.333333343267  Time elapse:  0:00:02.446472
Epoch 3 started Testing Set Accuracy: 0.416666671634  Time elapse:  0:00:02.519480
Epoch 4 started Testing Set Accuracy: 0.333333335817  Time elapse:  0:00:02.557558
Epoch 5 started Testing Set Accuracy: 0.500000007451  Time elapse:  0:00:02.424104
Epoch 6 started Testing Set Accuracy: 0.166666671634  Time elapse:  0:00:02.542757
Epoch 7 started Testing Set Accuracy: 0.250000007451  Time elapse:  0:00:02.480380
Epoch 8 started Testing Set Accuracy: 0.500000007451  Time elapse:  0:00:02.425845
Epoch 9 started Testing Set Accuracy: 0.0833333358169  Time elapse:  0:00:02.265855
Time elapse:  0:00:24.519134
